In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/mimic-cxr-reports.zip -d /content/drive/MyDrive/mimic-cxr-reports


Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/mimic-cxr-reports/files/p11/p11633410/s55390414.txt  
  inflating: /content/drive/MyDrive/mimic-cxr-reports/files/p11/p11633410/s54144044.txt  
  inflating: /content/drive/MyDrive/mimic-cxr-reports/files/p11/p11633410/s56238888.txt  
   creating: /content/drive/MyDrive/mimic-cxr-reports/files/p11/p11371772/
  inflating: /content/drive/MyDrive/mimic-cxr-reports/files/p11/p11371772/s58048952.txt  
   creating: /content/drive/MyDrive/mimic-cxr-reports/files/p11/p11995273/
  inflating: /content/drive/MyDrive/mimic-cxr-reports/files/p11/p11995273/s55294910.txt  
   creating: /content/drive/MyDrive/mimic-cxr-reports/files/p11/p11262628/
  inflating: /content/drive/MyDrive/mimic-cxr-reports/files/p11/p11262628/s56016122.txt  
   creating: /content/drive/MyDrive/mimic-cxr-reports/files/p11/p11253657/
  inflating: /content/drive/MyDrive/mimic-cxr-reports/files/p11/p11253657/s51065216.txt  
  inflating: /conte

In [ ]:
import tensorflow
import keras


In [ ]:
!pip install Keras-Preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.8 MB/s eta 0:00:00


In [ ]:
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku

from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

import pandas as pd
import numpy as np
import string, os

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
curr_dir = '/content/p10_copy/p10_copy'
all_report_data = []

for files in os.listdir(curr_dir):
    dir = os.path.join(curr_dir, files)

    for txt_file in os.listdir(dir):
        txt_dir = os.path.join(dir, txt_file)

        with open(txt_dir, 'r') as txt_file:
            text_data = txt_file.read()
            all_report_data.append(text_data)

In [ ]:
all_report_data

[' WET READ: ___ ___ ___ 1:36 PM\n  \n  1. Widened mediastinum may reflect low lung volumes and supine positioning,\n  however mediastinal great vessel injury cannot be excluded. If concern exists,\n  recommend chest CTA.\n  2. Pulmonary vascular congestion and probable mild pulmonary edema.\n  3. Bibasilar atelectasis.\n  4. ET tube with tip 6 cm above carina. Enteric tube with tip projecting over\n  stomach fundus.\n ______________________________________________________________________________\n                                 FINAL REPORT\n EXAMINATION:  AP chest x-ray.\n \n INDICATION:  A ___-year-old man following a cardiac arrest status post\n intubation, evaluate ET tube placement.\n \n TECHNIQUE:  Supine AP chest radiograph.\n \n COMPARISON:  None.\n \n FINDINGS: \n \n An ET tube is seen with distal tip projecting 6 cm above the carina, in\n appropriate position.  An enteric tube is seen coursing inferiorly, with\n distal tip projecting over the expected position of stomach fu

In [ ]:
import string
import re

def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    txt = re.sub(' +', ' ', txt)
    return txt.strip()

corpus = [clean_text(x) for x in all_report_data]
print(corpus[:10])

['wet read 136 pm\n \n 1 widened mediastinum may reflect low lung volumes and supine positioning\n however mediastinal great vessel injury cannot be excluded if concern exists\n recommend chest cta\n 2 pulmonary vascular congestion and probable mild pulmonary edema\n 3 bibasilar atelectasis\n 4 et tube with tip 6 cm above carina enteric tube with tip projecting over\n stomach fundus\n \n final report\n examination ap chest xray\n \n indication a yearold man following a cardiac arrest status post\n intubation evaluate et tube placement\n \n technique supine ap chest radiograph\n \n comparison none\n \n findings \n \n an et tube is seen with distal tip projecting 6 cm above the carina in\n appropriate position an enteric tube is seen coursing inferiorly with\n distal tip projecting over the expected position of stomach fundus\n \n there are low lung volumes a widened mediastinum may reflect low lung volumes\n and supine positioning pulmonary vascular engorgement and diffuse airspace\n op

In [ ]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)
inp_sequences[:10]

[[140, 134],
 [140, 134, 1685],
 [140, 134, 1685, 141],
 [140, 134, 1685, 141, 107],
 [140, 134, 1685, 141, 107, 519],
 [140, 134, 1685, 141, 107, 519, 226],
 [140, 134, 1685, 141, 107, 519, 226, 136],
 [140, 134, 1685, 141, 107, 519, 226, 136, 334],
 [140, 134, 1685, 141, 107, 519, 226, 136, 334, 94],
 [140, 134, 1685, 141, 107, 519, 226, 136, 334, 94, 26]]

In [ ]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [ ]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()

    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))

    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))

    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # Added accuracy metric

    return model

model = create_model(max_sequence_len, total_words)
model.summary()
model.fit(predictors, label, epochs=50, verbose=5)


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 246, 10)           28090     
                                                                 
 lstm_5 (LSTM)               (None, 100)               44400     
                                                                 
 dropout_5 (Dropout)         (None, 100)               0         
                                                                 
 dense_5 (Dense)             (None, 2809)              283709    
                                                                 
Total params: 356,199
Trainable params: 356,199
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50


KeyboardInterrupt: ignored

In [ ]:
import numpy as np

def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1)[0]

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text.title()


In [ ]:
print (generate_text(-ffusion", 5, model, max_sequence_len))


Pleural Effusion Pulmonary Pulmonary Edema With Persistent


In [ ]:
model.save('LSTM_model.h5')

In [ ]:
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from numpy.random import seed
import pandas as pd
import numpy as np
import string, os, re
import warnings
from keras.layers import Bidirectional
from keras.optimizers import Adam

seed(1)
from tensorflow import random
random.set_seed(2)

warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

curr_dir = '/content/p10_copy/p10_copy'
all_report_data = []

for files in os.listdir(curr_dir):
    dir = os.path.join(curr_dir, files)

    for txt_file in os.listdir(dir):
        txt_dir = os.path.join(dir, txt_file)

        with open(txt_dir, 'r') as txt_file:
            text_data = txt_file.read()
            all_report_data.append(text_data)

def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii", 'ignore')
    txt = re.sub(' +', ' ', txt)
    return txt.strip()

corpus = [clean_text(x) for x in all_report_data]

tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)

def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    predictors, label = input_sequences[:, :-1], input_sequences[:, -1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

predictors_train, predictors_test, label_train, label_test = train_test_split(predictors, label, test_size=0.2, random_state=42)

def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()

    # Increase Embedding Dimension
    model.add(Embedding(total_words, 50, input_length=input_len))

    # Add More LSTM Layers with More Units and Bidirectional LSTM
    model.add(Bidirectional(LSTM(150, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(LSTM(100))
    model.add(Dropout(0.2))

    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    # Use Adam Optimizer with Different Learning Rate
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return model

model = create_model(max_sequence_len, total_words)
history = model.fit(predictors_train, label_train, epochs=10, verbose=1, validation_data=(predictors_test, label_test))


plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

model.save('LSTM_model.h5')

def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1)[0]
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text.title()

print(generate_text("fusion", 5, model, max_sequence_len))


Epoch 1/100
 424/1505 [=======>......................] - ETA: 24:42 - loss: 6.0688 - accuracy: 0.0500

KeyboardInterrupt: ignored

In [ ]:
import numpy as np
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from keras_preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import keras.utils as ku
from sklearn.model_selection import train_test_split
import os, re, string
from keras.initializers import Constant

def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

glove_file_path = '/content/glove.6B.300d.txt'
glove_embeddings = load_glove_embeddings(glove_file_path)

curr_dir = '/content/mimic-cxr-reports/mimic-cxr-reports/files'
all_report_data = []

for folder1 in os.listdir(curr_dir):
    folder1_path = os.path.join(curr_dir, folder1)
    for folder2 in os.listdir(folder1_path):
        folder2_path = os.path.join(folder1_path, folder2)
        for txt_file in os.listdir(folder2_path):
            txt_path = os.path.join(folder2_path, txt_file)
            if os.path.isfile(txt_path):
                with open(txt_path, 'r') as file:
                    text_data = file.read()
                    all_report_data.append(text_data)

def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii", 'ignore')
    txt = re.sub(' +', ' ', txt)
    return txt.strip()

corpus = [clean_text(x) for x in all_report_data]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

embedding_dim = 300
embedding_matrix = np.zeros((total_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = glove_embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label = ku.to_categorical(label, num_classes=total_words)

predictors_train, predictors_test, label_train, label_test = train_test_split(predictors, label, test_size=0.2, random_state=42)

def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    model.add(Embedding(total_words, embedding_dim, embeddings_initializer=Constant(embedding_matrix), input_length=input_len, trainable=False))
    model.add(LSTM(100))
    model.add(Dropout(0.2))
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return model


model = create_model(max_sequence_len, total_words)
history = model.fit(predictors_train, label_train, epochs=30, verbose=1, validation_data=(predictors_test, label_test))

model.save('LSTM_model.h5')

def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1)[0]
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text.title()

user_query = input("Enter your query: ")
generated_text = generate_text(user_query, 5, model, max_sequence_len)
print("Generated text:", generated_text)


Epoch 1/30
2329/2329 [==============================] - 58s 22ms/step - loss: 5.0079 - accuracy: 0.1784 - val_loss: 4.1514 - val_accuracy: 0.2777
Epoch 2/30
2329/2329 [==============================] - 52s 22ms/step - loss: 3.8341 - accuracy: 0.3138 - val_loss: 3.6607 - val_accuracy: 0.3495
Epoch 3/30
2329/2329 [==============================] - 51s 22ms/step - loss: 3.4270 - accuracy: 0.3560 - val_loss: 3.4614 - val_accuracy: 0.3727
Epoch 4/30
2329/2329 [==============================] - 51s 22ms/step - loss: 3.1812 - accuracy: 0.3784 - val_loss: 3.3652 - val_accuracy: 0.3856
Epoch 5/30
2329/2329 [==============================] - 51s 22ms/step - loss: 3.0072 - accuracy: 0.3935 - val_loss: 3.3031 - val_accuracy: 0.3966
Epoch 6/30
2329/2329 [==============================] - 56s 24ms/step - loss: 2.8663 - accuracy: 0.4080 - val_loss: 3.2590 - val_accuracy: 0.4050
Epoch 7/30
2329/2329 [==============================] - 51s 22ms/step - loss: 2.7569 - accuracy: 0.4171 - val_loss: 3.2388 -

In [ ]:
loaded_model = load_model('LSTM_model.h5')
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1)[0]
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text.title()

while True:
    user_query = input("Enter your query (type 'exit' to quit): ")
    if user_query.strip().lower() == 'exit':
        break
    generated_text = generate_text(user_query, 100, model, max_sequence_len) # Generating 100 words
    print("Generated text:", generated_text)



Generated text: Effusion Urgent Reaches Report Chest Radiograph Indication Year Old Man With Chf And Acute Renal Failure Impression In Comparison With The Study Of The Monitoring And Support Devices Are Essentially Unchanged Continued Enlargement Of The Cardiac Silhouette With Elevated Pulmonary Venous Pressure And Bilateral Pleural Effusions With Underlying Compressive Atelectasis More Prominent On The Left Lung Base No Pleural Effusion Or Pneumothorax Is Seen The Cardiac Silhouette Is Normal The Mediastinal And Hilar Contours Are Normal There Is No Pleural Effusion Or Pneumothorax Conclusion There Is No Pleural Effusion Or Pneumothorax Impression No Acute Cardiopulmonary Process Specifically No Evidence Of Pneumonia
Generated text: Details Of Effusion Cardiac Silhouette With No Acute Osseous Abnormalities Impression No Acute Cardiopulmonary Process No Evidence Of Pneumonia Or Pneumonia No Evidence Of Pneumonia Or Pneumonia No Pneumonia Vascular Congestion Or Pleural Effusion No Pneum

In [ ]:
import numpy as np
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from keras_preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import keras.utils as ku
from sklearn.model_selection import train_test_split
import os, re, string
from keras.initializers import Constant
from keras.callbacks import EarlyStopping

def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

glove_file_path = '/content/glove.6B.300d.txt'
glove_embeddings = load_glove_embeddings(glove_file_path)

curr_dir = '/content/mimic-cxr-reports/mimic-cxr-reports/files'
all_report_data = []
for folder1 in os.listdir(curr_dir):
    folder1_path = os.path.join(curr_dir, folder1)
    for folder2 in os.listdir(folder1_path):
        folder2_path = os.path.join(folder1_path, folder2)
        for txt_file in os.listdir(folder2_path):
            txt_path = os.path.join(folder2_path, txt_file)
            if os.path.isfile(txt_path):
                with open(txt_path, 'r') as file:
                    text_data = file.read()
                    all_report_data.append(text_data)

def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii", 'ignore')
    txt = re.sub(' +', ' ', txt)
    return txt.strip()

corpus = [clean_text(x) for x in all_report_data]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

embedding_dim = 300
embedding_matrix = np.zeros((total_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = glove_embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label = ku.to_categorical(label, num_classes=total_words)

predictors_train, predictors_test, label_train, label_test = train_test_split(predictors, label, test_size=0.2, random_state=42)

def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    model.add(Embedding(total_words, embedding_dim, embeddings_initializer=Constant(embedding_matrix), input_length=input_len, trainable=False))
    model.add(Bidirectional(LSTM(150, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(LSTM(100))
    model.add(Dropout(0.2))
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return model

model = create_model(max_sequence_len, total_words)

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(predictors_train, label_train, epochs=50, batch_size=64, verbose=1, validation_data=(predictors_test, label_test), callbacks=[early_stopping])

model.save('LSTM_model1.h5')




Epoch 1/50
2130/2329 [==========================>...] - ETA: 12s - loss: 5.0883 - accuracy: 0.1692

KeyboardInterrupt: ignored

In [ ]:
loaded_model = load_model('LSTM_model1.h5')

In [ ]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1)[0]
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text.title()

while True:
    user_query = input("Enter your query (type 'exit' to quit): ")
    if user_query.strip().lower() == 'exit':
        break
    generated_text = generate_text(user_query, 100, model, max_sequence_len) # Generating 100 words
    print("Generated text:", generated_text)

Enter your query (type 'exit' to quit): effusion
Generated text: Effusion Is Seen With New Opacity In The Right Lung Base Is Unchanged In Position Of The Right Lung Base Is Likely Due To Atelectasis Or Aspiration Or Aspiration Or Aspiration Or Aspiration There Is No Pneumothorax Or Pleural Effusion Is Seen Impression No Acute Cardiopulmonary Process No Evidence Of Acute Cardiopulmonary Process No Evidence Of Acute Cardiopulmonary Process No Evidence Of Acute Cardiopulmonary Process Final Report Examination Chest Pa And Lat Indication Year Old Woman With Cough And Cough And Cough And Cough And Cough And Cough And Cough And Cough And Shortness Of Breath Evaluate For Pneumonia Comparison Chest


In [ ]:
import numpy as np
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from keras_preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import keras.utils as ku
from sklearn.model_selection import train_test_split
import os, re, string
from keras.initializers import Constant
from keras.callbacks import EarlyStopping

def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

glove_file_path = '/content/glove.6B.300d.txt'
glove_embeddings = load_glove_embeddings(glove_file_path)

curr_dir = '/content/mimic-cxr-reports/mimic-cxr-reports/files'
all_report_data = []
for folder1 in os.listdir(curr_dir):
    folder1_path = os.path.join(curr_dir, folder1)
    for folder2 in os.listdir(folder1_path):
        folder2_path = os.path.join(folder1_path, folder2)
        for txt_file in os.listdir(folder2_path):
            txt_path = os.path.join(folder2_path, txt_file)
            if os.path.isfile(txt_path):  # Check if it's a file
                with open(txt_path, 'r') as file:
                    text_data = file.read()
                    all_report_data.append(text_data)

def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii", 'ignore')
    txt = re.sub(' +', ' ', txt)
    return txt.strip()

corpus = [clean_text(x) for x in all_report_data]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

embedding_dim = 300
embedding_matrix = np.zeros((total_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = glove_embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label = ku.to_categorical(label, num_classes=total_words)

predictors_train, predictors_test, label_train, label_test = train_test_split(predictors, label, test_size=0.2, random_state=42)

def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    model.add(Embedding(total_words, embedding_dim, embeddings_initializer=Constant(embedding_matrix), input_length=input_len, trainable=False))
    model.add(Bidirectional(LSTM(150, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(LSTM(100))
    model.add(Dropout(0.2))
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return model

model = create_model(max_sequence_len, total_words)

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)

history = model.fit(predictors_train, label_train, epochs=40, batch_size=32, verbose=1, validation_data=(predictors_test, label_test), callbacks=[early_stopping])

model.save('LSTM_model32early.h5')




Epoch 1/40
2329/2329 [==============================] - 162s 68ms/step - loss: 5.0635 - accuracy: 0.1711 - val_loss: 4.2011 - val_accuracy: 0.2862
Epoch 2/40
2329/2329 [==============================] - 162s 69ms/step - loss: 3.8822 - accuracy: 0.3115 - val_loss: 3.6891 - val_accuracy: 0.3457
Epoch 3/40
2329/2329 [==============================] - 162s 70ms/step - loss: 3.4471 - accuracy: 0.3576 - val_loss: 3.4585 - val_accuracy: 0.3740
Epoch 4/40
2329/2329 [==============================] - 163s 70ms/step - loss: 3.1836 - accuracy: 0.3808 - val_loss: 3.3454 - val_accuracy: 0.3940
Epoch 5/40
2329/2329 [==============================] - 162s 70ms/step - loss: 2.9898 - accuracy: 0.3993 - val_loss: 3.2813 - val_accuracy: 0.4076
Epoch 6/40
2329/2329 [==============================] - 162s 70ms/step - loss: 2.8385 - accuracy: 0.4131 - val_loss: 3.2605 - val_accuracy: 0.4120
Epoch 7/40
2329/2329 [==============================] - 162s 70ms/step - loss: 2.7124 - accuracy: 0.4251 - val_loss: 3

In [ ]:
loaded_model = load_model('LSTM_model32early.h5')

In [ ]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1)[0]
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text.title()

while True:
    user_query = input("Enter your query (type 'exit' to quit): ")
    if user_query.strip().lower() == 'exit':
        break
    generated_text = generate_text(user_query, 100, model, max_sequence_len) # Generating 100 words
    print("Generated text:", generated_text)

Generated text: Effusion Findings Were Discussed With Dr By Dr By Dr By Dr By Telephone On At Pm 5 Minutes After Discovery Of The Findings Were Discussed By Dr By Dr At The Telephone On At Time Of Radiology Critical Communications System For Direct Communication To The Referring Provider By Dr At The Telephone On At Time Of Radiology Critical Communications System For Direct Communication To The Referring Provider By Dr By Dr At The Telephone On At Time Of Radiology Critical Communications System For Direct Communication To The Referring Provider By Dr By Dr At The Telephone On At Time Of
Enter your query (type 'exit' to quit): exit


In [ ]:
!pip install nltk



In [ ]:
import os
import re
import numpy as np
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.initializers import Constant
from keras.callbacks import EarlyStopping
import keras.utils as ku
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index
from nltk.corpus import wordnet

# Function to augment the data using synonyms
def augment_data(sentence):
    words = sentence.split()
    new_words = words.copy()
    for i, word in enumerate(words):
        if word not in stop_words:
            synonyms = wordnet.synsets(word)
            if synonyms:
                synonym = synonyms[0].lemmas()[0].name()
                new_words[i] = synonym
    return ' '.join(new_words)

def clean_text(txt):
    txt = re.sub(r'(final report|examination|indication|technique|comparison|findings|impression):', '', txt, flags=re.IGNORECASE)
    txt = re.sub(r'\[.*?\]', ' ', txt)
    txt = re.sub(r'___.*?___', ' ', txt)
    txt = re.sub(r'[^a-zA-Z ]', ' ', txt)
    txt = txt.lower()
    txt = ' '.join([lemmatizer.lemmatize(word) for word in txt.split() if word not in stop_words and len(word) > 1])
    txt = re.sub(r' +', ' ', txt)
    return txt.strip()

def read_files(directory):
   for folder1 in os.listdir(directory):
    folder1_path = os.path.join(directory, folder1)
    for folder2 in os.listdir(folder1_path):
        folder2_path = os.path.join(folder1_path, folder2)
        for txt_file in os.listdir(folder2_path):
            txt_path = os.path.join(folder2_path, txt_file)
            if os.path.isfile(txt_path):  # Check if it's a file
                with open(txt_path, 'r') as file:
                    text_data = file.read()
                    all_report_data.append(text_data)


from keras.layers import Bidirectional
from keras.regularizers import l2

def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    model.add(Embedding(total_words, embedding_dim, embeddings_initializer=Constant(embedding_matrix), input_length=input_len, trainable=True))
    model.add(Bidirectional(LSTM(100, return_sequences=True)))
    model.add(LSTM(100))
    model.add(Dropout(0.3))
    model.add(Dense(total_words, activation='softmax', kernel_regularizer=l2(0.001))) # Reduced regularization
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return model


glove_file_path = '/content/drive/MyDrive/glove.6B/glove.6B.300d.txt'
directory ='/content/drive/MyDrive/mimic-cxr-reports1111/files'

# Load Embeddings
glove_embeddings = load_glove_embeddings(glove_file_path)

# Reading and preprocessing data
all_report_data= []
read_files(directory)
# Apply cleaning and augmentation
corpus = [clean_text(x) for x in all_report_data]
augmented_corpus = [augment_data(x) for x in corpus]
corpus.extend(augmented_corpus)
# Print the first 5 samples of original and augmented data
num_samples_to_print = min(5, len(corpus))
for i in range(num_samples_to_print):
    print(f"Original Sample {i + 1}: {corpus[i]}")
    print(f"Augmented Sample {i + 1}: {augmented_corpus[i]}")

# Tokenization and sequence preparation
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

embedding_dim = 300
embedding_matrix = np.zeros((total_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = glove_embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label = ku.to_categorical(label, num_classes=total_words)

predictors_train, predictors_test, label_train, label_test = train_test_split(predictors, label, test_size=0.2, random_state=42)

model = create_model(max_sequence_len, total_words)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(predictors_train, label_train, epochs=30, batch_size=32, verbose=1, validation_data=(predictors_test, label_test), callbacks=[early_stopping])
model.save('LSTM_model_finetuned3layerbi.h5')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Original Sample 1: wet read pm right basilar atelectasis evidence focal consolidation final report chest pa lat history stroke stroke chest radiograph single frontal view chest obtained lung volume normal focal consolidation effusion pneumothorax linear opacity right lower lobe likely represent subsegmental atelectasis versus scarring heart enlarged unchanged right basilar atelectasis evidence focal consolidation
Augmented Sample 1: moisture read autopsy right basilar atelectasis evidence focal consolidation final report thorax dad latissimus_dorsi history stroke stroke thorax radiogram single frontlet position thorax obtain lung volume convention focal consolidation effusion pneumothorax linear opacity right lower_berth lobe likely represent subsegmental atelectasis versus scar heart enlarge unchanged right basilar atelectasis evidence focal consolidation
Original Sample 2: final report chest radiograph history altered mental status question aspiration comparison none chest portable a

In [ ]:
from keras.models import  load_model
loaded_model = load_model('LSTM_model_finetuned3layerbi.h5')

In [ ]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1)[0]
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text.title()

while True:
    user_query = input("Enter your query (type 'exit' to quit): ")
    if user_query.strip().lower() == 'exit':
        break
    generated_text = generate_text(user_query, 100, loaded_model, 342) # Generating 100 words
    print("Generated text:", generated_text)

Generated text: Lung Opacity                                                                                                    


In [ ]:
pip install keras

In [ ]:
import numpy as np
import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer


tokenizer = Tokenizer()

def generate_text(seed_text, next_words, model, max_sequence_len, temperature=1.0):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]

        # Apply temperature to the predicted probabilities
        predicted_probs = np.log(predicted_probs + 1e-7) / temperature
        exp_preds = np.exp(predicted_probs)
        predicted_probs = exp_preds / np.sum(exp_preds)

        # Sample the next word index from the adjusted probabilities
        predicted = np.random.choice(range(len(predicted_probs)), p=predicted_probs)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word

    return seed_text.title()


In [ ]:
while True:
    user_query = input("Enter your query (type 'exit' to quit): ")
    if user_query.strip().lower() == 'exit':
        break
    generated_text = generate_text(user_query, 100, loaded_model, 342, temperature=0.1)
    print("Generated text:", generated_text)




Enter your query (type 'exit' to quit): lung opacity?
Generated text: Lung Opacity?                                                                                                    
Enter your query (type 'exit' to quit): haha hehe
Generated text: Haha Hehe                                                                                                    


KeyboardInterrupt: ignored

In [ ]:
import os
import re
import numpy as np
from keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras.regularizers import l2
import keras.utils as ku
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.initializers import Constant
import nltk

nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

from nltk.corpus import wordnet

def augment_data(sentence):
    words = sentence.split()
    new_words = words.copy()
    for i, word in enumerate(words):
        if word not in stop_words:
            synonyms = wordnet.synsets(word)
            if synonyms:
                synonym = synonyms[0].lemmas()[0].name()
                new_words[i] = synonym
    return ' '.join(new_words)

def clean_text(txt):
    txt = re.sub(r'(final report|examination|indication|technique|comparison|findings|impression):', '', txt, flags=re.IGNORECASE)
    txt = re.sub(r'\[.*?\]', ' ', txt)
    txt = re.sub(r'___.*?___', ' ', txt)
    txt = re.sub(r'[^a-zA-Z ]', ' ', txt)
    txt = txt.lower()
    txt = ' '.join([lemmatizer.lemmatize(word) for word in txt.split() if word not in stop_words and len(word) > 1])
    txt = re.sub(r' +', ' ', txt)
    return txt.strip()

def read_files(directory):
    all_report_data = []
    for folder1 in os.listdir(directory):
        folder1_path = os.path.join(directory, folder1)
        for folder2 in os.listdir(folder1_path):
            folder2_path = os.path.join(folder1_path, folder2)
            for txt_file in os.listdir(folder2_path):
                txt_path = os.path.join(folder2_path, txt_file)
                if os.path.isfile(txt_path):  # Check if it's a file
                    with open(txt_path, 'r') as file:
                        text_data = file.read()
                        all_report_data.append(text_data)
    return all_report_data

def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    model.add(Embedding(total_words, embedding_dim, embeddings_initializer=Constant(embedding_matrix), input_length=input_len, trainable=True))
    model.add(LSTM(200, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(100, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(100))
    model.add(BatchNormalization())
    model.add(Dense(total_words, activation='softmax', kernel_regularizer=l2(0.001)))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return model

def lr_schedule(epoch):
    lr = 0.001
    if epoch > 10:
        lr *= 0.5e-3
    elif epoch > 7:
        lr *= 1e-3
    elif epoch > 5:
        lr *= 1e-2
    elif epoch > 3:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

# Paths
glove_file_path = '/content/drive/MyDrive/glove.6B/glove.6B.300d.txt'
directory ='/content/drive/MyDrive/mimic-cxr-reports/files'

# Preprocessing
glove_embeddings = load_glove_embeddings(glove_file_path)
all_report_data = read_files(directory)
corpus = [clean_text(x) for x in all_report_data]
augmented_corpus = [augment_data(x) for x in corpus]
corpus.extend(augmented_corpus)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
embedding_dim = 300
embedding_matrix = np.zeros((total_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = glove_embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label = ku.to_categorical(label, num_classes=total_words)
predictors_train, predictors_test, label_train, label_test = train_test_split(predictors, label, test_size=0.2, random_state=42)

model = create_model(max_sequence_len, total_words)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = LearningRateScheduler(lr_schedule)
history = model.fit(predictors_train, label_train, epochs=20, batch_size=64, verbose=1, validation_data=(predictors_test, label_test), callbacks=[early_stopping, lr_scheduler])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
